## Libraries

In [8]:
from pprint import pprint
import numpy as np

## Loading dataset

In [1]:
from datasets import load_dataset

datasets = load_dataset("squad")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to C:/Users/Gyanprakash/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
datasets['train'][45]

{'id': '5733afd3d058e614000b6047',
 'title': 'University_of_Notre_Dame',
 'context': 'In 2015-2016, Notre Dame ranked 18th overall among "national universities" in the United States in U.S. News & World Report\'s Best Colleges 2016. In 2014, USA Today ranked Notre Dame 10th overall for American universities based on data from College Factual. Forbes.com\'s America\'s Best Colleges ranks Notre Dame 13th among colleges in the United States in 2015, 8th among Research Universities, and 1st in the Midwest. U.S. News & World Report also lists Notre Dame Law School as 22nd overall. BusinessWeek ranks Mendoza College of Business undergraduate school as 1st overall. It ranks the MBA program as 20th overall. The Philosophical Gourmet Report ranks Notre Dame\'s graduate philosophy program as 15th nationally, while ARCHITECT Magazine ranked the undergraduate architecture program as 12th nationally. Additionally, the study abroad program ranks sixth in highest participation percentage in the natio

In [9]:
len(datasets['train'])

87599

In [20]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [24]:
def see_rand_qs(end,data = 'train'):
    if data == 'train':
        for i in range(0,end):
            indxs = np.random.randint(low=0,high=len(datasets['train']))
            pprint(datasets['train'][indxs])
            print()
    else:
        for i in range(0,end):
            indxs = np.random.randint(low=0,high=len(datasets['validation']))
            pprint(datasets['validation'][indxs])
            print()

            

In [50]:
see_rand_qs(1,data='val')

{'answers': {'answer_start': [253, 253, 253],
             'text': ['Inherited wealth',
                      'Inherited wealth',
                      'Inherited wealth']},
 'context': 'According to PolitiFact the top 400 richest Americans "have more '
            'wealth than half of all Americans combined." According to the New '
            'York Times on July 22, 2014, the "richest 1 percent in the United '
            'States now own more wealth than the bottom 90 percent". Inherited '
            'wealth may help explain why many Americans who have become rich '
            'may have had a "substantial head start". In September 2012, '
            'according to the Institute for Policy Studies, "over 60 percent" '
            'of the Forbes richest 400 Americans "grew up in substantial '
            'privilege".',
 'id': '5727e8424b864d1900163fc2',
 'question': 'What is considered as a potential advantage for wealth for some '
             'Americans?',
 'title': 'Economic_inequ

In [57]:
see_rand_qs(1,data='train')

{'answers': {'answer_start': [254], 'text': ['a storm']},
 'context': 'Winter, and a deteriorating supply situation on both sides of '
            'troops and materiel, led to a halt in ground operations. '
            'Sevastopol remained invested by the allies, while the allied '
            'armies were hemmed in by the Russian army in the interior. On 14 '
            'November a storm sank thirty allied transport ships including HMS '
            'Prince which was carrying a cargo of winter clothing.:435 The '
            'storm and heavy traffic caused the road from the coast to the '
            'troops to disintegrate into a quagmire, requiring engineers to '
            'devote most of their time to its repair including quarrying '
            'stone. A tramroad was ordered. It arrived in January with a '
            'civilian engineering crew, however it was March before it was '
            'sufficiently advanced to be of any appreciable value.:439 An '
            'Electric

## Preprocessing the training data


In [55]:
from transformers import AutoTokenizer

In [56]:
model_checkpoint = "distilbert-base-cased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

c:\Users\Gyanprakash\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Gyanprakash\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [58]:
max_length = 384  # The maximum length of a feature (question and context)
doc_stride = (
    128  # The authorized overlap between two part of the context when splitting
) # it is needed.

